In [17]:
from dotenv import load_dotenv
from portia.cli import CLIExecutionHooks
from portia import *
import requests
import xml.etree.ElementTree as ET
from pydantic import BaseModel, Field
from typing import Generic, TypeVar, List, ClassVar, Dict
from notion_client import Client
import os
load_dotenv(override=True)

# Fetch the Notion API key
notion_api_key = os.getenv("NOTION_API_KEY")
notion_parent_id = os.getenv("NOTION_PARENT_ID")

youtube_api_key = os.getenv("GOOGLE_API_KEY")

# Initialize the Notion client
notion = Client(auth=notion_api_key)


In [ ]:
topics = [
{
'topic': "Least Squares Regression",
'page_id': "1d3a9322-4732-81ce-a5a2-f3163cbfe28f"
},
{
'topic': "Ordinary Differential Equations (ODEs)",
'page_id': "1d3a9322-4732-81cb-849a-dddb95979b5f"
},
{
'topic': "Paper Summary",
'page_id': "1d3a9322-4732-813d-aa14-c6349117780d"
}
]

In [19]:
class PaperSummaryToolSchema(BaseModel):
    """Input schema for PaperSummaryTool."""
    topics: List[Dict[str, str]] = Field(..., description="List of topics and their associated Notion page IDs.")

class PaperSummaryTool(Tool[str]):
    """Creates a summary Notion subpage for the full paper with explanation and novelties."""

    id: ClassVar[str] = "paper_summary_tool"
    name: ClassVar[str] = "Paper Summary Tool"
    description: ClassVar[str] = "Creates a Notion subpage summarizing the full paper and its novelties."
    args_schema = PaperSummaryToolSchema
    output_schema: ClassVar[tuple[str, str]] = (
        "str",
        "Confirmation of the summary page creation."
    )
    def run(self, context: ToolRunContext, topics: List[Dict[str, str]]) -> str:
        import os
        from notion_client import Client

        notion = Client(auth=os.getenv("NOTION_API_KEY"))
        paper_topic = next((t for t in topics if t["topic"].lower() == "paper summary"), None)

        if not paper_topic:
            return "❌ 'Paper Summary' page not found in input."

        parent_id = paper_topic["page_id"]

        existing_blocks = notion.blocks.children.list(block_id=parent_id).get("results", [])
        for block in existing_blocks:
            try:
                notion.blocks.delete(block["id"])
            except:
                continue

        # Append blocks to the existing Paper Summary page
        notion.blocks.children.append(
            block_id=parent_id,
            children=[
                {
                    "object": "block",
                    "type": "heading_2",
                    "heading_2": {
                        "rich_text": [{"type": "text", "text": {"content": "🧠 Overview"}}]
                    }
                },
                {
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {"rich_text": [{"type": "text", "text": {"content": ""}}]}
                },
                {
                    "object": "block",
                    "type": "heading_2",
                    "heading_2": {
                        "rich_text": [{"type": "text", "text": {"content": "🌟 Novelties / Contributions"}}]
                    }
                },
                {
                    "object": "block",
                    "type": "bulleted_list_item",
                    "bulleted_list_item": {
                        "rich_text": [{"type": "text", "text": {"content": ""}}]
                    }
                },
                {
                    "object": "block",
                    "type": "heading_2",
                    "heading_2": {
                        "rich_text": [{"type": "text", "text": {"content": "🔗 Related Subtopics"}}]
                    }
                },
                {
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {"rich_text": [{"type": "text", "text": {"content": "To be linked..."}}]}
                }
            ]
        )

        return "✅ 'Paper Summary' page populated with initial structure."


In [ ]:
# class RecReadToolSchema(BaseModel): 

#     """Input Schema for RecReadTool."""
#     topics: List[Dict[str, str]] = Field(description="The list of topic pages and their IDs.")

# class RecReadTool(Tool[None]):

#     """A tool to find recommended reading for topics, and link them to Notion."""

#     id: ClassVar[str] = "RecRead_tool"
#     name: ClassVar[str] = "RecRead Tool"
#     description: ClassVar[str] = "A tool to find recommended reading for given topics, and link them to Notion."
#     args_schema = RecReadToolSchema
#     output_schema: ClassVar[tuple[str, str]] = (
#         "str",
#         "Confirmation of task completion."
#     )

#     def run(self, context, topics: List[Dict[str, str]]) -> None:
#         """Adds recommended Wikipedia reading to Notion pages for each topic."""
        
#         output = []

#         for topic in topics:
#             topic_name = topic["topic"]
#             page_id = topic["page_id"]

#             print(f"Processing topic: {topic_name}")

#             # Wikipedia API: Search
#             search_url = "https://en.wikipedia.org/w/api.php"
#             summary_url = "https://en.wikipedia.org/api/rest_v1/page/summary/"
#             search_params = {
#                 "action": "query",
#                 "list": "search",
#                 "srsearch": topic_name,
#                 "format": "json"
#             }

#             search_response = requests.get(search_url, params=search_params).json()
#             results = search_response.get("query", {}).get("search", [])

#             if not results:
#                 continue  # Skip this topic if nothing is found

#             best_title = results[0]["title"]

#             # Wikipedia API: Summary
#             summary_response = requests.get(f"{summary_url}{best_title.replace(' ', '_')}")
#             if summary_response.status_code != 200:
#                 continue

#             data = summary_response.json()
#             wiki_title = data.get("title")
#             wiki_summary = data.get("extract")
#             wiki_url = data["content_urls"]["desktop"]["page"]


#             ### Free Textbooks
#             google_books_url = f"https://www.googleapis.com/books/v1/volumes?q={topic_name.replace(' ', '+')}+textbook&maxResults=3"
#             books_response = requests.get(google_books_url).json()
#             textbooks = []

#             for book in books_response.get("items", []):
#                 info = book["volumeInfo"]
#                 dictionary = {
#                     "title": info.get("title"), 
#                     "authors": ", ".join(info.get("authors", [])), 
#                     "description": info.get("description", ""), 
#                     "link": info.get("infoLink")
#                 }
#                 textbooks.append(dictionary)

#             if len(textbooks) == 0: 
#                 textbooks_block = [{
#                         "object": "block",
#                         "type": "paragraph",
#                         "paragraph": {
#                             "rich_text": [
#                                 {
#                                     "type": "text",
#                                     "text": {"content": "No textbooks found for this topic."}
#                                 }
#                             ]
#                         }
#                     }]

#             else: 
#                 textbooks_block = []
#                 for book in textbooks: 
#                     link_block = {"object": "block", "type": "paragraph","paragraph": 
#                                   {
#                                       "rich_text": [
#                                           {
#                                               "type": "text",
#                                               "text": {
#                                                   "content": f"🔗 {book['title']} by {book['authors']}",
#                                                   "link": {"url": book['link']}
#                                                   }
#                                             }]
#                                     }}
#                     content_block = {"object": "block","type": "paragraph", 
#                                   "paragraph": {
#                                         "rich_text": [
#                                             {
#                                                 "type": "text",
#                                                 "text": {"content": f"{book['description'][:100]}"}
#                                             }]}}
#                     textbooks_block.append(link_block)
#                     textbooks_block.append(content_block)

#             ### Article / Papers


#             # Append blocks directly to Notion page
#             notion.blocks.children.append(
#                 block_id=page_id,
#                 children=[
#                     {
#                         "object": "block",
#                         "type": "heading_2",
#                         "heading_2": {
#                             "rich_text": [
#                                 {
#                                     "type": "text",
#                                     "text": {"content": "📘 Recommended Reading"}
#                                 }
#                             ]
#                         }
#                     },
#                     {
#                         "object": "block",
#                         "type": "paragraph",
#                         "paragraph": {
#                             "rich_text": [
#                                 {
#                                     "type": "text",
#                                     "text": {
#                                         "content": f"🔗 {wiki_title} (Wikipedia)",
#                                         "link": {"url": wiki_url}
#                                     }
#                                 }
#                             ]
#                         }
#                     },
#                     {
#                         "object": "block",
#                         "type": "paragraph",
#                         "paragraph": {
#                             "rich_text": [
#                                 {
#                                     "type": "text",
#                                     "text": {"content": f"{wiki_summary}"}
#                                 }
#                             ]
#                         }
#                     }, 
#                     {
#                         "object": "block",
#                         "type": "paragraph",
#                         "paragraph": {
#                             "rich_text": [
#                                 {
#                                     "type": "text",
#                                     "text": {"content": f"📚 Textbooks"}
#                                 }
#                             ]
#                         }},
#                 ] + textbooks_block 
#             )

#             output.append(f"Added reading for '{topic_name}'")

#         return "✅ Recommended Reading added to Notion pages successfully."


In [20]:
tool = PaperSummaryTool()
tool.run(None, topics)

APIResponseError: Could not find block with ID: 1d3a9322-4732-81a3-b394-f205ee0770da. Make sure the relevant pages and databases are shared with your integration.